## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Analyze the Datasets

In [2]:
movies = pd.read_csv('../Datasets/Movies/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
movies.columns = ["Id", "Title", "Category"]
movies.head()

,Id,Title,Category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
users = pd.read_csv('../Datasets/Movies/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
users.columns = ["Id", "Gender", "Age", "Job", "Code"]
users.head()

,Id,Gender,Age,Job,Code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [4]:
ratings = pd.read_csv('../Datasets/Movies/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings.columns = ["UserId", "MovieId", "Rating", "DateTime"]
ratings.head()

,UserId,MovieId,Rating,DateTime
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# Data Preprocessing

## Prepare Training and Test Set

In [5]:
training_set_df = pd.read_csv('../Datasets/Movies/ml-100k/u1.base', delimiter='\t')
training_set_df.columns = ["UserId", "MovieId", "Rating", "TimeStamp"]
training_set_df.head()

,UserId,MovieId,Rating,TimeStamp
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [6]:
training_set = np.array(training_set_df, dtype='int')

In [7]:
test_set_df = pd.read_csv('../Datasets/Movies/ml-100k/u1.test', delimiter='\t')
test_set_df.columns = ["UserId", "MovieId", "Rating", "TimeStamp"]
test_set_df.head()

,UserId,MovieId,Rating,TimeStamp
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [8]:
test_set = np.array(test_set_df, dtype='int')

## Get total number of Users and Movies

In [9]:
n_total_users = max(max(training_set_df["UserId"].values), max(test_set_df["UserId"].values))
n_total_movies = max(max(training_set_df["MovieId"].values), max(test_set_df["MovieId"].values))
print("total users:", n_total_users, ",total movies:", n_total_movies) 

total users: 943 ,total movies: 1682


## Convert Data into Matrix with users in rows and movies in columns

In [10]:
def convert_to_matrix(data, total_users, total_movies):
    ratings_data = np.zeros(shape=(total_users, total_movies))
    for user_id, movie_id, rating, timestamp in data:
        ratings_data[user_id-1][movie_id-1] = rating
    return ratings_data

In [11]:
training_set = convert_to_matrix(training_set, n_total_users, n_total_movies)
test_set = convert_to_matrix(test_set, n_total_users, n_total_movies)

## Convert Data into Torch Tensors

In [12]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Convert Into Binary Ratings (Like/Dislike)

In [13]:
# convert non existant ratings to -1 (no rating for specific movie)
training_set[training_set<=0] = -1
# convert to not liked if rating is one or two
training_set[training_set==1] = 0
training_set[training_set==2] = 0
# convert movies with rating above or equal to 3 to liked
training_set[training_set>=3] = 1

test_set[test_set<=0] = -1
test_set[test_set==1] = 0
test_set[test_set==2] = 0
test_set[test_set>=3] = 1

# Build Restricted Boltzmann Machine

In [14]:
class RBM:
    def __init__(self, n_visible_nodes, n_hidden_nodes):
        self.weights = torch.randn(n_hidden_nodes, n_visible_nodes) # initialize weights with normal distribution
        self.bias_hidden = torch.randn(1, n_hidden_nodes)
        self.bias_visible = torch.randn(1, n_visible_nodes)
    
    def train(self, input_vector, visible_nodes_k_iters, hidden_probs_first_iter, hidden_probs_k_iters):
        """
        Train RBM Using Contrastive Divergence Technique
        """
        self.weights += (torch.mm(input_vector.t(), hidden_probs_first_iter) - torch.mm(visible_nodes_k_iters.t(), hidden_probs_k_iters)).t()
        self.bias_visible += torch.sum((input_vector - visible_nodes_k_iters), 0)
        self.bias_hidden +=  torch.sum((hidden_probs_first_iter - hidden_probs_k_iters), 0)
    
    def run_hidden(self, visible_nodes):
        """
        compute probability of hidden nodes given visible nodes
        """
        hidden_states = torch.mm(visible_nodes, self.weights.t())
        hidden_activations = hidden_states + self.bias_hidden.expand_as(hidden_states)
        hidden_probabilities = torch.sigmoid(hidden_activations)
        return hidden_probabilities, torch.bernoulli(hidden_probabilities)
    
    def run_visible(self, hidden_nodes):
        """
        compute probability of visible nodes given hidden nodes
        """
        visible_states = torch.mm(hidden_nodes, self.weights)
        visible_activations = visible_states + self.bias_visible.expand_as(visible_states)
        visible_probabilities = torch.sigmoid(visible_activations)
        return visible_probabilities, torch.bernoulli(visible_probabilities)

# Train RBM

In [15]:
rbm = RBM(n_total_movies, 100)

In [16]:
nb_epochs = 10
batch_size = 100
for epoch in range(1, nb_epochs+1):
    train_loss = 0
    counter = 0.0
    for user_id in range(0, n_total_users-batch_size, batch_size):
        visible_states_k = training_set[user_id : user_id+batch_size]
        input_states = training_set[user_id : user_id+batch_size]
        initial_hidden_probabilities, _ = rbm.run_hidden(input_states)
        for k in range(10):
            _, hidden_states_k = rbm.run_hidden(visible_states_k)
            _, visible_states_k = rbm.run_visible(hidden_states_k)
            # make sure unrated movies keep their -1 (not rated) state
            visible_states_k[input_states<0] = input_states[input_states<0]
        hidden_probabilities_k_iters, _ = rbm.run_hidden(visible_states_k)
        rbm.train(input_states, visible_states_k, initial_hidden_probabilities, hidden_probabilities_k_iters)
        train_loss += torch.mean(torch.abs(input_states[input_states>=0] - visible_states_k[input_states>=0]))
        counter+=1.0
    print("Epoch:", epoch, " loss:", train_loss/counter)

Epoch: 1  loss: tensor(0.3455)
Epoch: 2  loss: tensor(0.2342)
Epoch: 3  loss: tensor(0.2499)
Epoch: 4  loss: tensor(0.2483)
Epoch: 5  loss: tensor(0.2467)
Epoch: 6  loss: tensor(0.2507)
Epoch: 7  loss: tensor(0.2479)
Epoch: 8  loss: tensor(0.2478)
Epoch: 9  loss: tensor(0.2507)
Epoch: 10  loss: tensor(0.2459)


# Test RBM

In [17]:
test_loss = 0
counter = 0.0
for user_id in range(n_total_users):
    visible_states = training_set[user_id : user_id+1]
    input_state = test_set[user_id : user_id+1]
    initial_hidden_probabilities, _ = rbm.run_hidden(input_state)
    if (len(input_state[input_state>=0]) > 0):
        _, hidden_states = rbm.run_hidden(visible_states)
        _, visible_states = rbm.run_visible(hidden_states)
        test_loss += torch.mean(torch.abs(input_state[input_state>=0] - visible_states[input_state>=0]))
        counter+=1.0
print("test loss: ", test_loss/counter)

test loss:  tensor(0.2345)
